## Create a CWAS gene set: Step 1

### 0. Preparation

In [ ]:
# Import statements
import os
import yaml
import numpy as np
import pandas as pd

from collections import defaultdict

In [ ]:
# Path settings
project_dir = os.path.dirname(os.path.abspath('.'))
conf_dir = os.path.join(project_dir, 'conf3')
path_conf_path = os.path.join(conf_dir, 'filepaths.yaml')
gene_list_conf_path = os.path.join(conf_dir, 'gene_list_names.yaml')
gene_type_conf_path = os.path.join(conf_dir, 'gene_biotype.yaml')

with open(path_conf_path) as path_conf_file:
    path_dict = yaml.safe_load(path_conf_file)
    
with open(gene_list_conf_path) as gene_list_conf_file:
    gene_list_name_dict = yaml.safe_load(gene_list_conf_file)

log_dir = os.path.join(project_dir, path_dict['LOG_DIR'])
os.makedirs(log_dir, exist_ok=True)

gencode_path = os.path.join(project_dir, path_dict['GENCODE'])
prev_gene_mat_path = os.path.join(project_dir, path_dict['An2018'])
hgnc_path = os.path.join(project_dir, path_dict['HGNC'])
#alt_gene_list_path = os.path.join(project_dir, path_dict['An2018_ALT_GENE'])

In [ ]:
# Load the data
gencode_df = pd.read_table(gencode_path, compression='gzip', comment='#', names=['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])  
gencode_gene_df = gencode_df[gencode_df['feature'] == 'gene']  # List up only genes
gencode_tx_df = gencode_df[gencode_df['feature'] == 'transcript']  # List up only transcripts
hgnc_df = pd.read_table(hgnc_path, usecols=['hgnc_id', 'symbol', 'alias_symbol', 'prev_symbol', 'ensembl_gene_id', 'ucsc_id', 'refseq_accession'])
#prev_gene_mat_df = pd.read_excel(prev_gene_mat_path, sheet_name='8-1 Genesets')

In [ ]:
# Function to parse strings of the 'attribute' field in the GENCODE.
def parse_attr_str(attr_str):
    attrs = attr_str.split(';')
    attr_dict = {}
    
    for attr in attrs:
        key, value = attr.split('=')
        attr_dict[key] = value
        
    return attr_dict

In [ ]:
# Parse values of the 'attribute' field in the GENCODE
gene_to_attr_dict = {}  # Key: GeneID, Value: A dictionary for the information in the 'attribute' columns
gene_name_to_ids = defaultdict(list)  # Key: Gene name, Value: The list of gene IDs
hgnc_to_ids = defaultdict(list)  # Key: HGNC ID, Value: The list of gene IDs

for attr_str in gencode_gene_df['attribute'].values:
    attr_dict = parse_attr_str(attr_str)
    
    if not attr_dict['ID'].endswith('Y'):  # Ignore pseudoautosome region (PAR_Y)
        gene_id = attr_dict['ID'].split('.')[0]
        gene_to_attr_dict[gene_id] = attr_dict
        gene_name_to_ids[attr_dict['gene_name']].append(gene_id)
        hgnc_id = attr_dict.get('hgnc_id')
        
        if hgnc_id is not None:
            hgnc_to_ids[hgnc_id].append(gene_id)

tx_to_attr_dict = {}  # Key: GeneID, Value: A dictionary for the information in the 'attribute' columns

for attr_str in gencode_tx_df['attribute'].values:
    attr_dict = parse_attr_str(attr_str)
    
    if not attr_dict['ID'].endswith('Y'):  # Ignore pseudoautosome region (PAR_Y)
        tx_id = attr_dict['ID'].split('.')[0]
        tx_to_attr_dict[tx_id] = attr_dict

In [ ]:
# Parse the HGNC file and make a dictionary which key and value are a gene symbol and its ensembl gene ID, respectively.
hgnc_df_col_idx = {column: i for i, column in enumerate(hgnc_df.columns.values)}
hgnc_df_val = hgnc_df.values

symbol_to_gene_ids = {}
alias_to_gene_ids = {}
prev_symbol_to_gene_ids = {}

for hgnc_entry in hgnc_df_val:
    gene_symbol = hgnc_entry[hgnc_df_col_idx['symbol']]
    alias_symbol_str = hgnc_entry[hgnc_df_col_idx['alias_symbol']]
    prev_symbol_str = hgnc_entry[hgnc_df_col_idx['prev_symbol']]
    ensembl_gene_id = hgnc_entry[hgnc_df_col_idx['ensembl_gene_id']]
    
    if ensembl_gene_id is np.nan or gene_to_attr_dict.get(ensembl_gene_id) is None:
        gene_ids = None
    else:
        gencode_symbol = gene_to_attr_dict[ensembl_gene_id]['gene_name']
        gene_ids = gene_name_to_ids.get(gencode_symbol)  # From GENCODE v44
    
    if gene_ids is not None: 
        symbol_to_gene_ids[gene_symbol] = gene_ids

        if alias_symbol_str is not np.nan:
            alias_symbols = alias_symbol_str.split('|')

            for alias_symbol in alias_symbols:
                alias_to_gene_ids[alias_symbol] = gene_ids

        if prev_symbol_str is not np.nan:
            prev_symbols = prev_symbol_str.split('|')

            for prev_symbol in prev_symbols:
                prev_symbol_to_gene_ids[prev_symbol] = gene_ids

In [ ]:
# Function to find a gene ID from a gene symbol of the HGNC and the GENCODE v34
def find_gene_ids(gene_symbol):
    # Priority: 
    # 1. GENCODE v44
    # 2. HGNC: symbol -> alias -> previous symbol
    # If the gene IDs cannot be found, return None
    
    # GENCODE v44
    gene_ids = gene_name_to_ids.get(gene_symbol)
    
    # HGNC
    if gene_ids is None:
        gene_ids = symbol_to_gene_ids.get(gene_symbol)

        if gene_ids is None:
            gene_ids = alias_to_gene_ids.get(gene_symbol)

            if gene_ids is None:
                gene_ids = prev_symbol_to_gene_ids.get(gene_symbol)

    return gene_ids

In [ ]:
#gene_to_attr_dict.get('ENSG00000223972', dict())

In [ ]:
# Make a dictionary for the new gene matrix
new_gene_mat_dict = {}

for gene_id in gene_to_attr_dict: ## Edited
    gene_val_dict = {}
    gene_val_dict['gene_id'] = gene_to_attr_dict[gene_id]['ID']
    gene_val_dict['gene_name'] = gene_to_attr_dict[gene_id]['gene_name']        
    new_gene_mat_dict[gene_id] = gene_val_dict

# Make a DataFrame for the new gene matrix
gene_mat_df = pd.DataFrame.from_dict(new_gene_mat_dict, orient='index')
gene_mat_cols = list(gene_mat_df.columns.values)
gene_mat_cols = gene_mat_cols[-2:] + gene_mat_cols[:-2]
gene_mat_df = gene_mat_df[gene_mat_cols]
gene_mat_df.fillna(0, inplace=True)
gene_mat_df = gene_mat_df.astype({gene_list_col: 'int64' for gene_list_col in gene_mat_cols[2:]})
gene_mat_df.head()

In [ ]:
#a = {1,2,3,4,5,6}
#a = list(a)
#a[-2:] + a[:-2]

### 3. Add GENCODE biotypes as gene list columns into the gene matrix

In [ ]:
# Add GENCODE biotypes
with open(gene_type_conf_path) as gene_type_conf_file:
    biotype_dict = yaml.safe_load(gene_type_conf_file)

for biotype_category in biotype_dict:
    biotype_set = set(biotype_dict[biotype_category])
    gene_mat_val_dict = {gene_id: 1 if gene_to_attr_dict[gene_id]['gene_type'] in biotype_set else 0 for gene_id in gene_to_attr_dict}
    gene_mat_df[biotype_category] = pd.Series(gene_mat_val_dict)
    
gene_mat_df.head()

##### 03. Haploinsufficient genes

In [ ]:
# load the dataset
gnomad_gene_list_path = os.path.join(project_dir, path_dict['GNOMAD_GENE'])
gnomad_gene_df = pd.read_table(gnomad_gene_list_path)

# Extract HI genes (HI: Haploinsufficient)
is_hi_gene_func = lambda pli: pli is not np.nan and pli >= 0.9
is_hi_gene = np.vectorize(is_hi_gene_func)(gnomad_gene_df['pLI'].values)
hi_gene_df = gnomad_gene_df[is_hi_gene]  

# Dictionaries for pLI scores
tx_to_pli = {}
symbol_to_pli = {}
gene_to_pli = {}

# Update the 'tx_to_pli' and the 'symbol_to_pli'
hi_txs = hi_gene_df['transcript'].values
hi_symbols = hi_gene_df['gene'].values
hi_plis = hi_gene_df['pLI'].values

for tx_id, symbol, pli_score in zip(hi_txs, hi_symbols, hi_plis):
    tx_to_pli[tx_id] = pli_score
    prev_symbol_pli = symbol_to_pli.get(symbol)
    
    # Choose the maximum pLI score for duplicated symbols.
    if prev_symbol_pli is None or prev_symbol_pli < pli_score:  
        symbol_to_pli[symbol] = pli_score

# There are 3 steps to update the 'gene_to_pli'
# Step 1: Update the 'gene_to_pli' dictionary by getting gene IDs of the transcript IDs via the GENCODE
depr_tx_set = set()
hi_tx_list = hi_gene_df['transcript'].values

for tx_id in hi_tx_list:
    attr_dict = tx_to_attr_dict.get(tx_id)
    
    if attr_dict is None:
        depr_tx_set.add(tx_id)
    else:
        gene_id = attr_dict['Parent'].split('.')[0]
        prev_gene_pli = gene_to_pli.get(gene_id)
        pli_score = tx_to_pli[tx_id]
        
        # Choose the maximum pLI score for duplicated genes.
        if prev_gene_pli is None or prev_gene_pli < pli_score:    
            gene_to_pli[gene_id] = pli_score

# Step 2: Update the 'gene_to_pli' dictionary by getting gene IDs of the deprecated transcript IDs using 
depr_hi_gene_df = hi_gene_df[np.vectorize(lambda tx_id: tx_id in depr_tx_set)(hi_txs)]
depr_gene_symbols = depr_hi_gene_df['gene'].values

for depr_gene_symbol in depr_gene_symbols:
    gene_ids = find_gene_ids(depr_gene_symbol)
    
    if gene_ids is None:
        print(f'{depr_gene_symbol} cannot be found in both the GENCODE and the HGNC.')
        continue
    
    # Update only if the same gene ID does not exist
    for gene_id in gene_ids:
        if gene_to_pli.get(gene_id) is None:  
            gene_to_pli[gene_id] = symbol_to_pli[depr_gene_symbol]

# Update the gene matrix
# HC: High-confident
is_hi_gene = np.vectorize(lambda gene_id: 0 if gene_to_pli.get(gene_id) is None else 1)(gene_mat_df.index.values)  # pLI score >= 0.9
is_hc_hi_gene = np.vectorize(lambda gene_id: 1 if gene_to_pli.get(gene_id) is not None and gene_to_pli.get(gene_id) >= 0.995 else 0)(gene_mat_df.index.values)  # pLI score >= 0.995
#gene_mat_df[gene_list_name_dict['GNOMAD_PLI90']] = is_hi_gene
gene_mat_df[gene_list_name_dict['GNOMAD_PLI995']] = is_hc_hi_gene
gene_mat_df.head()

### 5. Save the gene matrix as a text file

In [ ]:
# TODO: Write your own file path in here
my_gene_mat_path = os.path.join(project_dir, 'my_gene_matrix_231123.txt')
gene_mat_df.to_csv(my_gene_mat_path, sep='\t', index=False)